##### Подготовка тестовых данных для тестирования

Порядок:

-  Подготовка Подготовка тестового массива для сравнения рассчитанных фичей с витриной ЦАЛ 
-  Формирование и запись вспомогательной таблицы pre_contracts: "все договоры ИНН на день принятия решения"
-  Расчет фичей 
-  Копирование данных в директорию "data/interm/test" 


In [19]:
pre = '../../' # путь до главной директории проекта
%run ../../params.ipynb #параметры как логин пароль к БИ, параметры расчёта ( в нашем случае их нет)
%run ../init.ipynb #подгрузить нужные библиотеки

In [20]:
# сюда таблицы служебные будем кидать, для отчетов
name = pre + "data/interm/auxilary/test1_target_features.xlsx"
writer = ExcelWriter(name)

In [ ]:
name = pre + "data/interm/target_data.pkl"
target_data = pd.read_pickle(name, compression={'method':'zip'})

print(len(target_data))
# target_data.head(1)

In [22]:
name = pre + "data/external/model_tsal_for_test.pkl"
model_tsal_prod = pd.read_pickle(name, compression={'method':'zip'})

print(len(model_tsal_prod))
# model_tsal_prod.head(1)

156409


,DAILY_SNAPSHOT_DT,NEW_INN,CONTRACT_CNT,DSNC_OVD_PAY_DAYS,CONTRACT_OVD_CNT,INSERT_DT,MAX_LEAS_OVD_PAY_DT
0,2023-07-04,2310189350,1,1408.0,1,2023-07-04 09:17:54,2019-08-26


In [23]:
print(model_tsal_prod['DAILY_SNAPSHOT_DT'].min())
print(model_tsal_prod['DAILY_SNAPSHOT_DT'].max())

2023-07-04 00:00:00
2023-07-05 00:00:00


In [24]:
# берем последний срез
model_tsal_prod = model_tsal_prod[model_tsal_prod['DAILY_SNAPSHOT_DT']==model_tsal_prod['DAILY_SNAPSHOT_DT'].max()]

num, duplicates = identify_duplicates(model_tsal_prod, 'NEW_INN')
Check_dupl(num, duplicates)

In [26]:
# берем последний срез
target_features_lst = target_data.drop_duplicates(subset='ИНН контрагента', keep='last').reset_index(drop=True)

num, duplicates = identify_duplicates(target_features_lst, 'ИНН контрагента')
Check_dupl(num, duplicates)

In [ ]:
target_data_test= target_features_lst.copy()
target_data_test['Дата принятия решения по сделке'] = model_tsal_prod['DAILY_SNAPSHOT_DT'].max()
target_data_test['decision_date'] = model_tsal_prod['DAILY_SNAPSHOT_DT'].max()
# target_data_test.head(1)

Записываем его, как будто принятие решения по сделкам 2023-07-05

In [28]:
keep_columns = ['PL', 'Номер сделки', 'Номер договора (кратко)', 'ИНН контрагента', 'Продукт',
                'Дата принятия решения по сделке', 'decision_date', 
                 'Метка кред дефолта']

target_data_test[keep_columns].to_pickle(pre+"data/interm/target_data.pkl", compression={'method':'zip'})

In [ ]:
%run ../preprocessing/contracts_of_inn.ipynb

после этого необходимо рассчитать все фичи 
скопировать файлы в "data/interm/test" 

In [ ]:
%run ../features/11okved.ipynb
%run ../features/12contracts.ipynb
%run ../features/13contract_ovd.ipynb
%run ../features/14leas_ovd_pay_dt.ipynb
%run ../features/21features_combine.ipynb

In [31]:
for f in ['feature_okved.pkl', 'leas_ovd_pay_dt.pkl', 'lst_contracts.pkl', 'overdue_contracts.pkl', 'target_data.pkl', 'target_features.pkl']:
    name = pre + "data/interm/" + f
    name_new = pre + "data/interm/test/" + f
    
    os.replace(name, name_new)

In [32]:
# сохраняем tmp таблицы
if len(writer.sheets):
    writer.save() 